In [ ]:
!pip install numpy==1.26.4 --force-reinstall
!pip install --no-cache-dir matplotlib==3.9.2
!pip install --no-cache-dir ultralytics --upgrade
!pip install mlflow==2.22.0

In [ ]:
import os
import mlflow

from ultralytics import YOLO   # ✅ Import obligatoire
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import mlflow
import json
from pathlib import Path

# Définir le chemin vers le fichier Ultralytics settings
settings_path = Path.home() / ".config" / "Ultralytics" / "settings.json"

# Charger les paramètres existants
with open(settings_path, "r") as f:
    settings = json.load(f)

# Mettre à jour pour activer MLflow
settings["mlflow"] = True

# Sauvegarder les paramètres
with open(settings_path, "w") as f:
    json.dump(settings, f, indent=4)

In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TensorFlow")

#The following code assumes that you have already_clsinstalled the necessary_clslibraries and have your data loaded into a pandas DataFrame called df_test.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Use the first GPU (T4), if you have more than one GPU, use the appropriate index.
#If you have only_clsone GPU, you can omit this line

# ... (Your existing code for data preprocessing and splitting)

In [ ]:
import os
import mlflow
import json
from pathlib import Path
from ultralytics import YOLO
import tensorflow as tf
import shutil
import warnings
warnings.filterwarnings("ignore")

# ----------------------------
# 1️⃣ Activer MLflow dans YOLO
# ----------------------------
settings_path = Path.home() / ".config" / "Ultralytics" / "settings.json"

# Charger les paramètres existants
with open(settings_path, "r") as f:
    settings = json.load(f)

# Activer MLflow
settings["mlflow"] = True

# Sauvegarder les paramètres
with open(settings_path, "w") as f:
    json.dump(settings, f, indent=4)

# ----------------------------
# 2️⃣ Vérifier le GPU
# ----------------------------
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TensorFlow")

# Sélectionner le GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ----------------------------
# 3️⃣ Configurer MLflow
# ----------------------------
mlflow.set_tracking_uri('https://tracking.ai.sigservice-dz.com/')
settings.update({"mlflow": True})

try:
    os.environ["MLFLOW_EXPERIMENT_NAME"] = 'model Yolo Danone2'
    os.environ["MLFLOW_RUN"] = "25/12/25/"
    print(f"MLflow experiment set to: 'model Yolo'")
except Exception as e:
    print(f"Warning: Could not set MLflow experiment: {e}")


# ----------------------------
# 5️⃣ Charger le modèle YOLO
# ----------------------------
print("Initializing YOLO model for classification training...")
model = YOLO("yolo11x-cls.pt")
print("YOLOv11 classification model loaded successfully")

# ----------------------------
# 6️⃣ Configurer l'entraînement
# ----------------------------
train_config = {
    'data': '/kaggle/input/nv-data/nv_val/data_nv_val',

    # ⏱️ Entraînement
    'epochs': 20,

    # 🖼️ Images plus petites → plus difficile
    'imgsz': 128,

    #  Learning rate 
    'lr0': 0.001,
    'lrf': 0.01,

    # Augmentations plus réalistes
    'auto_augment': 'autoaugment',
    'scale': 0.4,
    'fliplr': 0.3,
    'erasing': 0.2,
    'hsv_h': 0.02,
    'hsv_s': 0.3,
    'hsv_v': 0.3,

    # 🧯 Régularisation
    'weight_decay': 0.001,
    'dropout': 0.3,

    # ⚙️ Performance
    'batch': 512,
    'device': 0.1,
    'workers': 8,

    # 💾 MLflow
    'name': 'model_Yolo_danone_realistic',
    'save': True,
    'verbose': True,
    'plots': True,
}


# Afficher la configuration
print("Starting training with configuration:")
for key, value in train_config.items():
    print(f"  {key}: {value}")

print("STARTING YOLO TRAINING")
print("=" * 60)

# ----------------------------
# 7️⃣ Lancer l'entraînement
# ----------------------------
results = model.train(**train_config)
print("Training script completed successfully!")
